# Comparing BBG SWPM Swaption Pricing with `rvcore`

In [1]:
import sys
sys.path.append("../")

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

In [13]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
from core.utils.ql_loader import ql
from datetime import datetime
from typing import List

from core.IRSwaps import IRSwaps, IRSwapQuery, IRSwapValue, IRSwapStructure, IRSwapStructureFunctionMap, IRSwapValueFunctionMap
from core.IRSwaptions import IRSwaptions, IRSwaptionQuery, IRSwaptionValue, IRSwaptionStructure, IRSwaptionStructureFunctionMap, IRSwaptionValueFunctionMap
from core.DataFetching.FixingsFetcher import FixingsFetcher
from core.utils.ql_utils import ql_date_to_datetime

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.dates as mdates
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (12, 8),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
curve = "USD-SOFR-1D"
swaps_data_source = rf"CSV_{os.path.join(os.getcwd(), "..", "data", "USD-SOFR-1D_par_rates.csv")}"
swaptions_data_source = rf"CSV_{os.path.join(os.getcwd(), "..", "data", "usd_vol_cube.csv")}" 

usd_ois = IRSwaps(
    curve=curve,
    data_source=swaps_data_source,
    ql_interpolation_algo="log_linear",
    # pre_fetch_curves=True,
    error_verbose=True,
    max_njobs=-1,
)

usd_swaptions = IRSwaptions(
    data_source=swaptions_data_source,
    irswaps_product=usd_ois,
    error_verbose=True,
    max_n_jobs=-1,
)

In [8]:
usd_swaptions.fetch_qlcubes(bdates=[datetime(2025, 5, 15)], to_pydt=False)

{Timestamp('2025-05-15 21:00:00+0000', tz='UTC'): <QuantLib.QuantLib.InterpolatedSwaptionVolatilityCube; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedSwaptionVolatilityCube > *' at 0x000001A9D7FD2160> >}

In [15]:
usd_swaptions.fetch_scubes(bdates=[datetime(2025, 5, 15)], to_pydt=False)

{Timestamp('2025-05-15 21:00:00+0000', tz='UTC'): {-200: Tail            1Y          2Y          3Y          4Y          5Y  \
  Expiry                                                               
  1M      209.422199  200.459093  195.383658  191.865962  188.362984   
  3M      166.131387  161.648504  159.076743  155.175784  151.274826   
  6M      143.340139  140.936260  139.545705  135.547238  131.548771   
  9M      137.789996  135.357495  133.951387  129.771605  125.770133   
  1Y      132.421017  129.960645  128.539492  124.359710  120.179928   
  2Y       89.265999  103.763342  113.334447  112.889382  103.134199   
  3Y       99.265066   98.521240   98.087744   97.655433   97.223122   
  4Y       96.692857   95.948912   95.515417   94.967396   94.304297   
  5Y       94.521733   93.376584   92.711669   92.048571   91.385472   
  6Y       93.080463   91.773667   91.016068   90.352969   89.689871   
  7Y       91.581987   90.166359   89.346603   88.652723   87.989624   
  8Y     

## Pricing 1mm 1Mx10Y Payer Swaption on 08/14/2024

![womp womp](../dump/8-14-2024-swaption-swpm.png)

### Grab market data (the curve and cube)

In [4]:
dt = datetime(2024, 8, 14)

ql_curve = usd_ois.fetch_ql_irswap_curves(bdates=[dt], to_pydt=True)[dt]
curve_handle = ql.YieldTermStructureHandle(ql_curve)

ql.Settings.instance().evaluationDate = curve_handle.referenceDate() 

ql_cube = usd_swaptions.fetch_qlcubes(bdates=[dt], to_pydt=True)[dt]
cube_handle = ql.SwaptionVolatilityStructureHandle(ql_cube)
cube_pricing_engine = ql.BachelierSwaptionEngine(curve_handle, cube_handle)

BUILDING QL VOL CUBES...: 100%|██████████| 1/1 [00:00<00:00, 8355.19cube/s]


### Let's check what the curve and cube looked like

In [5]:
usd_swaptions.vol_surface_plotter(date=dt, strike_offset=0, use_plotly=True)

TypeError: can't compare offset-naive and offset-aware datetimes

In [48]:
usd_ois.irswaps_term_structure_plotter(bdates=[dt], fwd_tenors=["0D"], use_plotly=True)

PRICING IRSWAPS...: 100%|██████████| 1/1 [00:00<00:00,  2.26it/s]


### Pricer Set Up

In [49]:
query = IRSwaptionQuery(
    expiry="1M",
    tail="10Y",
    structure=IRSwaptionStructure.PAYER,
	side="buy",
	value=[
        IRSwaptionValue.NVOL,
        IRSwaptionValue.SPOT_PREM,
        IRSwaptionValue.FWD_PREM,
        IRSwaptionValue.DELTA,
        IRSwaptionValue.DV01,
        IRSwaptionValue.GAMMA_01,
        IRSwaptionValue.VEGA_01,
        IRSwaptionValue.THETA_1D,
    ],
    structure_kwargs={"strike": "ATMF", "notional": 1_000_000},
)

package, rws = IRSwaptionStructureFunctionMap(curve=curve, curve_handle=curve_handle, pricing_engine=cube_pricing_engine, side=query.side).apply(
    structure=query.structure, expiry=query.expiry, tail=query.tail, **query.structure_kwargs
)

package: List[ql.Swaption] = package
rws: List[float] = rws

print(f"Exr/Eff/Mat Dates ", package[0].exercise().dates()[0], package[0].underlying().startDate(), package[0].underlying().maturityDate())
print(f"Strike", package[0].underlying().fixedRate() * 100)
print(f"Notional", package[0].underlying().nominal())
print(f"NPV", package[0].underlying().NPV())
print(f"BPV", package[0].underlying().fixedLegBPS())
print("=======")

value_func_mapper = IRSwaptionValueFunctionMap(
    package=package, risk_weights=rws, curve=curve, curve_handle=curve_handle, pricing_engine=cube_pricing_engine, swaption_structure=query.structure
)
for sv in query.value:
    print(f"{sv.name}: {value_func_mapper.apply(sv)}")

Exr/Eff/Mat Dates  September 16th, 2024 September 16th, 2024 September 18th, 2034
Strike 3.36490191000782
Notional 1000000.0
NPV 5.820766091346741e-11
BPV -843.1967714015816
NVOL: 104.2663104396129
SPOT_PREM: 1.0546140293749644
FWD_PREM: 1.0594657027673848
DELTA: -0.5000000000000009
DV01: -421.59838570079154
GAMMA_01: 10.67040618724198
VEGA_01: 101.84617437476533
THETA_1D: -161.019231070748


## Pricing 100mm 1Yx5Y Payer Swaption on 02/23/2024


![womp womp](../dump/2-23-2024-swaption-swpm.png)

In [50]:
dt = datetime(2024, 2, 23)

ql_curve = usd_ois.fetch_ql_irswap_curves(bdates=[dt], to_pydt=True)[dt]
curve_handle = ql.YieldTermStructureHandle(ql_curve)

ql.Settings.instance().evaluationDate = curve_handle.referenceDate() 

ql_cube = usd_swaptions.fetch_qlcubes(bdates=[dt], to_pydt=True)[dt]
cube_handle = ql.SwaptionVolatilityStructureHandle(ql_cube)
cube_pricing_engine = ql.BachelierSwaptionEngine(curve_handle, cube_handle)

usd_swaptions.vol_surface_plotter(date=dt, strike_offset=0, use_plotly=True)
usd_ois.irswaps_term_structure_plotter(bdates=[dt], fwd_tenors=["0D"], use_plotly=True)


query = IRSwaptionQuery(
    expiry="1Y",
    tail="5Y",
    structure=IRSwaptionStructure.PAYER,
	side="buy",
	value=[
        IRSwaptionValue.NVOL,
        IRSwaptionValue.SPOT_PREM,
        IRSwaptionValue.FWD_PREM,
        IRSwaptionValue.DELTA,
        IRSwaptionValue.DV01,
        IRSwaptionValue.GAMMA_01,
        IRSwaptionValue.VEGA_01,
        IRSwaptionValue.THETA_1D,
    ],
    structure_kwargs={"strike": "ATMF", "notional": 100_000_000},
)

package, rws = IRSwaptionStructureFunctionMap(curve=curve, curve_handle=curve_handle, pricing_engine=cube_pricing_engine, side=query.side).apply(
    structure=query.structure, expiry=query.expiry, tail=query.tail, **query.structure_kwargs
)

package: List[ql.Swaption] = package
rws: List[float] = rws

print(f"Exr/Eff/Mat Dates ", package[0].exercise().dates()[0], package[0].underlying().startDate(), package[0].underlying().maturityDate())
print(f"Strike", package[0].underlying().fixedRate() * 100)
print(f"Notional", package[0].underlying().nominal())
print(f"NPV", package[0].underlying().NPV())
print(f"BPV", package[0].underlying().fixedLegBPS())
print("=======")

value_func_mapper = IRSwaptionValueFunctionMap(
    package=package, risk_weights=rws, curve=curve, curve_handle=curve_handle, pricing_engine=cube_pricing_engine, swaption_structure=query.structure
)
for sv in query.value:
    print(f"{sv.name}: {value_func_mapper.apply(sv)}")

PRICING IRSWAPS...: 100%|██████████| 1/1 [00:00<00:00,  2.62it/s]


Exr/Eff/Mat Dates  February 24th, 2025 February 24th, 2025 February 25th, 2030
Strike 3.718559946884997
Notional 100000000.0
NPV 5.587935447692871e-09
BPV -43185.45413554485
NVOL: 119.0894786578155
SPOT_PREM: 2.057347014637463
FWD_PREM: 2.162832641531632
DELTA: -0.5000000000000004
DV01: -21592.72706777244
GAMMA_01: 144.25072209445534
VEGA_01: 17395.19648144175
THETA_1D: -8184.72220617719
